In [ ]:
import os

import rawutil as ru
from analytics.utils import get_car_query
from database.wintap_duckdb import WintapDuckDB, WintapDuckDBOptions

# initially, this will only work with a configured .env file in the top level directory.
data_path=os.getenv('DATAPATH')

# Initialize an in-memory db. Save reference in a variable and then set magic-duckdb environment. 
# Result is ability to use the same DB instance from python code and %dql/%%dql magics.
# Also create views for every top-level type found in the current dataset.
con=ru.init_db(data_path)
options = WintapDuckDBOptions(con, data_path)
wintap_duckdb = WintapDuckDB(options)

In [ ]:
from analytics.utils import get_car_analytics
from database.constants import PID_HASH, ANALYTICS_RESULTS_TABLE
from datetime import datetime

CAR_1 = "CAR-2014-11-004"
analytics = get_car_analytics()

#for analytic in analytics:

results = wintap_duckdb.query(analytics[CAR_1].query_string)
for _, row in results.iterrows():
    wintap_duckdb.insert_analytics_table(analytics[CAR_1], row[PID_HASH])

print(wintap_duckdb.query('select * from analytics_results limit 10'))

# store results
#wintap_duckdb.write_table(ANALYTICS_RESULTS_TABLE, datetime.now().strftime("%Y%m%d"))


In [ ]:
# what T&Ts are associated with an entry in the analytics table
query = f"select entity, analytic_id from {ANALYTICS_RESULTS_TABLE} limit 1"
result = wintap_duckdb.query(query)
for _, row in result.iterrows():
    analytic_id = row['analytic_id']
    print(f"Entity: {row['entity']}:")
    for entry in analytics[analytic_id].coverage:
        print(f"\tTactic: {entry.tactics}, Technique(s): {entry.technique}")